In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(application_df[['EIN', 'NAME']], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_types = application_df['APPLICATION_TYPE'].value_counts()
application_types

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_types[application_types < 700].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_1 = classification_counts.loc[classification_counts > 1]
classification_counts_1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 700].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_dummies.drop('IS_SUCCESSFUL', axis=1).values
y = application_dummies['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Optimization Part 1: Added more neurons to the second layer

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 80, input_dim = input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 50, input_dim = input_features, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 50)                4050      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 7621 (29.77 KB)
Trainable params: 7621 (29.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, validation_split=0.2)

Epoch 1/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5728 - accuracy: 0.7175 - val_loss: 0.5714 - val_accuracy: 0.7203
Epoch 2/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5540 - accuracy: 0.7299 - val_loss: 0.5699 - val_accuracy: 0.7193
Epoch 3/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5507 - accuracy: 0.7313 - val_loss: 0.5632 - val_accuracy: 0.7230
Epoch 4/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5483 - accuracy: 0.7327 - val_loss: 0.5677 - val_accuracy: 0.7258
Epoch 5/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5478 - accuracy: 0.7334 - val_loss: 0.5610 - val_accuracy: 0.7224
Epoch 6/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7351 - val_loss: 0.5629 - val_accuracy: 0.7232
Epoch 7/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5450 - accuracy: 0.7354 - val_loss: 0.5609 - val_accuracy: 0.7236

Epoch 58/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5338 - accuracy: 0.7449 - val_loss: 0.5686 - val_accuracy: 0.7246
Epoch 59/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5338 - accuracy: 0.7427 - val_loss: 0.5640 - val_accuracy: 0.7250
Epoch 60/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5339 - accuracy: 0.7417 - val_loss: 0.5676 - val_accuracy: 0.7240
Epoch 61/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5334 - accuracy: 0.7428 - val_loss: 0.5667 - val_accuracy: 0.7242
Epoch 62/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5334 - accuracy: 0.7421 - val_loss: 0.5685 - val_accuracy: 0.7240
Epoch 63/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5336 - accuracy: 0.7428 - val_loss: 0.5668 - val_accuracy: 0.7259
Epoch 64/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5334 - accuracy: 0.7429 - val_loss: 0.5677 - val_accuracy:

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5551 - accuracy: 0.7258 - 173ms/epoch - 646us/step
Loss: 0.5551331043243408, Accuracy: 0.7258309125900269


Optimization Part 2: Added another layer while keeping the increased neurons

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 80, input_dim = input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 50, input_dim = input_features, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = 30, input_dim = input_features, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3520      
                                                                 
 dense_4 (Dense)             (None, 50)                4050      
                                                                 
 dense_5 (Dense)             (None, 30)                1530      
                                                                 
 dense_6 (Dense)             (None, 1)                 31        
                                                                 
Total params: 9131 (35.67 KB)
Trainable params: 9131 (35.67 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, validation_split=0.2)

Epoch 1/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5684 - accuracy: 0.7245 - val_loss: 0.5661 - val_accuracy: 0.7203
Epoch 2/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5534 - accuracy: 0.7306 - val_loss: 0.5633 - val_accuracy: 0.7258
Epoch 3/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5489 - accuracy: 0.7343 - val_loss: 0.5663 - val_accuracy: 0.7230
Epoch 4/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5468 - accuracy: 0.7349 - val_loss: 0.5736 - val_accuracy: 0.7281
Epoch 5/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7363 - val_loss: 0.5613 - val_accuracy: 0.7236
Epoch 6/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5453 - accuracy: 0.7366 - val_loss: 0.5587 - val_accuracy: 0.7228
Epoch 7/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5449 - accuracy: 0.7348 - val_loss: 0.5595 - val_accuracy: 0.7261

Epoch 58/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5326 - accuracy: 0.7427 - val_loss: 0.5678 - val_accuracy: 0.7232
Epoch 59/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5319 - accuracy: 0.7439 - val_loss: 0.5702 - val_accuracy: 0.7219
Epoch 60/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5319 - accuracy: 0.7443 - val_loss: 0.5724 - val_accuracy: 0.7213
Epoch 61/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5318 - accuracy: 0.7435 - val_loss: 0.5690 - val_accuracy: 0.7240
Epoch 62/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5318 - accuracy: 0.7442 - val_loss: 0.5702 - val_accuracy: 0.7232
Epoch 63/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5315 - accuracy: 0.7439 - val_loss: 0.5715 - val_accuracy: 0.7236
Epoch 64/100
644/644 [==============================] - 1s 1ms/step - loss: 0.5317 - accuracy: 0.7428 - val_loss: 0.5718 - val_accuracy:

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5680 - accuracy: 0.7269 - 180ms/epoch - 672us/step
Loss: 0.5680056214332581, Accuracy: 0.7268804907798767


Optimization Part 3: Doubled the epochs while keeping the increased layers and neurons

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 80, input_dim = input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 50, input_dim = input_features, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = 30, input_dim = input_features, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                3520      
                                                                 
 dense_8 (Dense)             (None, 50)                4050      
                                                                 
 dense_9 (Dense)             (None, 30)                1530      
                                                                 
 dense_10 (Dense)            (None, 1)                 31        
                                                                 
Total params: 9131 (35.67 KB)
Trainable params: 9131 (35.67 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200, validation_split=0.2)

Epoch 1/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5717 - accuracy: 0.7202 - val_loss: 0.5632 - val_accuracy: 0.7238
Epoch 2/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5524 - accuracy: 0.7320 - val_loss: 0.5683 - val_accuracy: 0.7252
Epoch 3/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5502 - accuracy: 0.7331 - val_loss: 0.5620 - val_accuracy: 0.7228
Epoch 4/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5475 - accuracy: 0.7362 - val_loss: 0.5605 - val_accuracy: 0.7213
Epoch 5/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7331 - val_loss: 0.5578 - val_accuracy: 0.7238
Epoch 6/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5439 - accuracy: 0.7345 - val_loss: 0.5626 - val_accuracy: 0.7240
Epoch 7/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5444 - accuracy: 0.7366 - val_loss: 0.5603 - val_accuracy: 0.7221

Epoch 58/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5331 - accuracy: 0.7436 - val_loss: 0.5650 - val_accuracy: 0.7252
Epoch 59/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5331 - accuracy: 0.7415 - val_loss: 0.5664 - val_accuracy: 0.7232
Epoch 60/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5321 - accuracy: 0.7435 - val_loss: 0.5647 - val_accuracy: 0.7236
Epoch 61/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5321 - accuracy: 0.7439 - val_loss: 0.5666 - val_accuracy: 0.7248
Epoch 62/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5316 - accuracy: 0.7442 - val_loss: 0.5703 - val_accuracy: 0.7246
Epoch 63/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5321 - accuracy: 0.7429 - val_loss: 0.5690 - val_accuracy: 0.7242
Epoch 64/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5319 - accuracy: 0.7420 - val_loss: 0.5647 - val_accuracy:

Epoch 115/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5293 - accuracy: 0.7447 - val_loss: 0.5879 - val_accuracy: 0.7228
Epoch 116/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5291 - accuracy: 0.7440 - val_loss: 0.5844 - val_accuracy: 0.7238
Epoch 117/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5285 - accuracy: 0.7439 - val_loss: 0.5851 - val_accuracy: 0.7234
Epoch 118/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5287 - accuracy: 0.7440 - val_loss: 0.5887 - val_accuracy: 0.7232
Epoch 119/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5287 - accuracy: 0.7437 - val_loss: 0.5887 - val_accuracy: 0.7228
Epoch 120/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5288 - accuracy: 0.7443 - val_loss: 0.5834 - val_accuracy: 0.7224
Epoch 121/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5284 - accuracy: 0.7444 - val_loss: 0.5904 - val_ac

644/644 [==============================] - 1s 1ms/step - loss: 0.5272 - accuracy: 0.7451 - val_loss: 0.6297 - val_accuracy: 0.7232
Epoch 172/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5278 - accuracy: 0.7444 - val_loss: 0.6154 - val_accuracy: 0.7232
Epoch 173/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5284 - accuracy: 0.7451 - val_loss: 0.6058 - val_accuracy: 0.7228
Epoch 174/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5274 - accuracy: 0.7451 - val_loss: 0.6085 - val_accuracy: 0.7234
Epoch 175/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5269 - accuracy: 0.7454 - val_loss: 0.6147 - val_accuracy: 0.7232
Epoch 176/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5273 - accuracy: 0.7447 - val_loss: 0.6141 - val_accuracy: 0.7224
Epoch 177/200
644/644 [==============================] - 1s 1ms/step - loss: 0.5275 - accuracy: 0.7448 - val_loss: 0.6130 - val_accuracy: 0.7224

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5829 - accuracy: 0.7237 - 179ms/epoch - 669us/step
Loss: 0.5829212069511414, Accuracy: 0.7237317562103271


In [24]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

C:\Users\jdaiu\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
